In [45]:
import os
import csv
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Convolution2D, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Cropping2D

samples = []
with open('/Users/mwk1/Desktop/laps/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
with open('/Users/mwk1/Desktop/corrections/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [46]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center, left, right, center_angle, left_angle, right_angle = images_and_angles(batch_sample)
                images.append(center)
                images.append(left)
                images.append(right)
                images.append(np.fliplr(center))
                images.append(np.fliplr(left))
                images.append(np.fliplr(right))
                angles.append(center_angle)
                angles.append(left_angle)
                angles.append(right_angle)
                angles.append(-center_angle)
                angles.append(-left_angle)
                angles.append(-right_angle)
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

def images_and_angles(batch_sample, correction = 0.25):
    center = cv2.imread(batch_sample[0])
    left = cv2.imread(batch_sample[1])
    right = cv2.imread(batch_sample[2])
    
    center_angle = float(batch_sample[3])
    left_angle = float(batch_sample[3]) + correction
    right_angle = float(batch_sample[3]) - correction

    return center, left, right, center_angle, left_angle, right_angle

In [47]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [48]:
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
model.add(Convolution2D(36,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
model.add(Convolution2D(48,5,5,border_mode='valid', activation='relu', subsample=(2,2)))
model.add(Convolution2D(64,3,3,border_mode='valid', activation='relu', subsample=(1,1)))
model.add(Convolution2D(64,3,3,border_mode='valid', activation='relu', subsample=(1,1)))
model.add(Flatten())
model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='mse', optimizer='adam')

In [49]:
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), nb_epoch=10)

Epoch 1/10
5952/6064 [============================>.] - ETA: 1s - loss: 0.0424

/Users/mwk1/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6144/6064 [==============================] - 74s - loss: 0.0418 - val_loss: 0.0280
Epoch 2/10
6144/6064 [==============================] - 72s - loss: 0.0283 - val_loss: 0.0250
Epoch 3/10
6144/6064 [==============================] - 73s - loss: 0.0256 - val_loss: 0.0232
Epoch 4/10
6144/6064 [==============================] - 69s - loss: 0.0224 - val_loss: 0.0199
Epoch 5/10
6144/6064 [==============================] - 69s - loss: 0.0194 - val_loss: 0.0212
Epoch 6/10
6240/6064 [==============================] - 72s - loss: 0.0201 - val_loss: 0.0211
Epoch 7/10
6144/6064 [==============================] - 70s - loss: 0.0194 - val_loss: 0.0197
Epoch 8/10
6144/6064 [==============================] - 72s - loss: 0.0206 - val_loss: 0.0216
Epoch 9/10
6144/6064 [==============================] - 72s - loss: 0.0211 - val_loss: 0.0184
Epoch 10/10
6144/6064 [==============================] - 69s - loss: 0.0181 - val_loss: 0.0172


In [50]:
model.save('more-epochs.h5')